In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split,StratifiedKFold,KFold
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV,ElasticNet, Lasso,  BayesianRidge, LinearRegression, RANSACRegressor
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, BaggingRegressor, AdaBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures

import warnings
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

pd.set_option('max_columns',1000) 
pd.set_option('max_row',300)  
#pd.set_option(‘display.float_format’, lambda x: ‘%.5f’ % x)

In [ ]:
#读取
train_ = pd.read_csv(r'D:\\BaiduNetdiskDownload\\Data\\二手车\\train_2.csv')
test_ = pd.read_csv(r'D:\\BaiduNetdiskDownload\\Data\\二手车\\test_2.csv')
feature = pd.read_csv(r'D:\\BaiduNetdiskDownload\\Data\\二手车\\lgb_feature.csv')

In [ ]:
#catboost调参 330mae562  520   3000 508   10000 520,508
#330dep12 560
#  RMSE, MultiRMSE, MAE, Quantile, LogLinQuantile, Poisson, MAPE, Lq
model_cb = cb.CatBoostRegressor(iterations=330,learning_rate=0.1,depth=10,loss_function='MAE')
#lgb 524

score = []
kfolder = KFold(n_splits=3,shuffle=True,random_state=100)
for q,(train_index, test_index) in enumerate(kfolder.split(train_[feature],train_['price'])):
    time1 = time.time()
    Xtrain, Ytrain = train_.loc[train_index,feature],train_.loc[train_index,'price']
    Xtest, Ytest = train_.loc[test_index,feature],train_.loc[test_index,'price']

    cb_reg = model_cb.fit(Xtrain,Ytrain)
    score_r = model_cb.score(Xtest,Ytest)
    t = time.time() - time1
    #print(k)
    print("lcb:{}".format(score_r))
    y_pre = cb_reg.predict(Xtest)
    print(abs(np.expm1(y_pre) - np.expm1(Ytest)).mean())
    print(t,'秒')
    score.append(abs(np.expm1(y_pre) - np.expm1(Ytest)).mean())
print("MAE平均:",np.mean(score))
print('')

In [ ]:
#xgb调参
for k in range(2500,3500,500):
    model_xgb = xgb.XGBRegressor(n_estimators=2500 ,learning_rate=0.1 ,random_state=420
                   ,booster='dart' ,nthread=12 ,eval_metric='mae',max_depth=6,reg_lambda=1)
    
    time1 = time.time()
    score = []
    #Xtrain, Xtest, Ytrain, Ytest = train_test_split(train_[feature],train_['price'],test_size=1/3)
    kfolder = KFold(n_splits=5,shuffle=True,random_state=100)
    for q,(train_index, test_index) in enumerate(kfolder.split(train_[feature],train_['price'])):
        
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(train_.loc[test_index,feature],train_.loc[test_index,'price'],test_size=1/3)

        xgb_reg = model_xgb.fit(Xtrain,Ytrain)
        score_r = model_xgb.score(Xtest,Ytest)
        t = time.time() - time1
        print(k)
        print("xgb:{}".format(score_r))
        y_pre = xgb_reg.predict(Xtest)
        print(abs(y_pre - Ytest).mean())
        print(t,'秒')
        score.append(abs(y_pre - Ytest).mean())
    print(k,":",np.mean(score))
    print('')

In [ ]:
#rf调参
for k in ['mae','friedman_mse','mse']:
    rfr = RandomForestRegressor(random_state=0,n_estimators=100,n_jobs=-1,criterion=k)

    score = []
    #Xtrain, Xtest, Ytrain, Ytest = train_test_split(train_[feature],train_['price'],test_size=1/3)
    kfolder = KFold(n_splits=3,shuffle=True,random_state=100)
    for q,(train_index, test_index) in enumerate(kfolder.split(train_[feature],train_['price'])):
        time1 = time.time()
        Xtrain, Ytrain = train_.loc[train_index,feature],train_.loc[train_index,'price']
        Xtest, Ytest = train_.loc[test_index,feature],train_.loc[test_index,'price']
        
        rf_reg = rfr.fit(Xtrain,Ytrain)
        score_r = rfr.score(Xtest,Ytest)
        t = time.time() - time1
        print(k)
        print("rf:{}".format(score_r))
        y_pre = rf_reg.predict(Xtest)
        print(abs(y_pre - Ytest).mean())
        print(t,'秒')
        score.append(abs(y_pre - Ytest).mean())
    print(k,"MAE平均:",np.mean(score))
    print('')